## 图像文件读取与标准化

In [34]:
import torch
import numpy as np

In [35]:
torch.set_printoptions(edgeitems=2, threshold=50)

新版本imageio分为了v3和v2两个命名空间，直接使用imageio.imread会调用v3的版本，但是为了使老版本v2版本的依旧可用，并消除DeprecationWarning，可以：
- 在调包时使用`import imageio.v2 as iio`
- 或者直接使用`imageio.v2.imread`

**不过这里也没有与该函数强关联的代码，尝试使用v3版本的iio，看看有没有啥问题。**

In [36]:
# import imageio
# img_arr = imageio.imread("../../myData/p1ch4/image-dog/bobby.jpg")
import imageio.v3 as iio

In [37]:
img_arr = iio.imread("../../myData/p1ch4/image-dog/bobby.jpg")
img_arr.shape

(720, 1280, 3)

In [38]:
img_arr.dtype

dtype('uint8')

In [39]:
img = torch.from_numpy(img_arr)
img.shape, img.dtype

(torch.Size([720, 1280, 3]), torch.uint8)

In [40]:
out = img.permute(2, 0, 1)
out.shape, out.dtype

(torch.Size([3, 720, 1280]), torch.uint8)

In [41]:
test = torch.tensor(list(range(24)))
a = test.view(2, 4, 3)
a

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23]]])

In [42]:
c = a.transpose(0, 2)
c

tensor([[[ 0, 12],
         [ 3, 15],
         [ 6, 18],
         [ 9, 21]],

        [[ 1, 13],
         [ 4, 16],
         [ 7, 19],
         [10, 22]],

        [[ 2, 14],
         [ 5, 17],
         [ 8, 20],
         [11, 23]]])

In [43]:
b = a.permute(2, 0, 1)
b

tensor([[[ 0,  3,  6,  9],
         [12, 15, 18, 21]],

        [[ 1,  4,  7, 10],
         [13, 16, 19, 22]],

        [[ 2,  5,  8, 11],
         [14, 17, 20, 23]]])

In [44]:
c[0, 0, 0] = 99
a[0, 1, 2] = 999
a, b, c

(tensor([[[ 99,   1,   2],
          [  3,   4, 999],
          [  6,   7,   8],
          [  9,  10,  11]],
 
         [[ 12,  13,  14],
          [ 15,  16,  17],
          [ 18,  19,  20],
          [ 21,  22,  23]]]),
 tensor([[[ 99,   3,   6,   9],
          [ 12,  15,  18,  21]],
 
         [[  1,   4,   7,  10],
          [ 13,  16,  19,  22]],
 
         [[  2, 999,   8,  11],
          [ 14,  17,  20,  23]]]),
 tensor([[[ 99,  12],
          [  3,  15],
          [  6,  18],
          [  9,  21]],
 
         [[  1,  13],
          [  4,  16],
          [  7,  19],
          [ 10,  22]],
 
         [[  2,  14],
          [999,  17],
          [  8,  20],
          [ 11,  23]]]))

In [45]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [46]:
import os

In [47]:
data_dir = "../../myData/p1ch4/image-cats/"

In [48]:
filenames = [
    name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == ".png"
]

In [49]:
filenames

['cat1.png', 'cat2.png', 'cat3.png']

In [53]:
size_set = set()
for f in filenames:
    img_arr = iio.imread(os.path.join(data_dir, f))
    size_set.add(img_arr.shape)

for item in size_set:
    print(item)

(256, 256, 3)


In [50]:
for idx, filename in enumerate(filenames):
    img_arr = iio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    # 保险起见，有的图片可能存在更多的通道(alpha通道)，这里我们只需要取前三个通道
    img_t = img_t[:3]
    batch[idx] = img_t

In [54]:
batch = batch.to(dtype=torch.float)
batch /= 255.0

In [55]:
n_channels = batch.shape[1]

In [56]:
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

In [57]:
batch[:, c].std()

tensor(1.)